In [ ]:
import numpy as np
import cvxpy as cp
import numpy as np


def parse_linear_condition(lin_con):
    coef_matrix = lin_con[0]
    bnd_key, bl, bu = lin_con[1][0], lin_con[1][1], lin_con[1][2]
    fx_bnds_loc = [i for i in range(len(bnd_key)) if bnd_key[i] == 'fx']
    eq_matrix = coef_matrix[fx_bnds_loc, :].copy()
    eq_bnd = bl[fx_bnds_loc]
    #
    up_bl_bu_loc = [i for i in range(len(bnd_key)) if bnd_key[i] in ('ra', 'up')]
    lo_bl_bu_loc = [i for i in range(len(bnd_key)) if bnd_key[i] in ('ra', 'lo')]
    ineq_matrix = np.vstack((coef_matrix[up_bl_bu_loc, :], -coef_matrix[lo_bl_bu_loc, :]))
    ineq_bnd = np.hstack((bu[up_bl_bu_loc], -bl[lo_bl_bu_loc]))
    return ineq_matrix, ineq_bnd, eq_matrix, eq_bnd


def parse_bound_conditions(bnd_con, bnd_inf):
    assert isinstance(bnd_inf, float) and bnd_inf > 0.0
    bnd_key, lb, ub = bnd_con
    lb[bnd_key == 'up'] = -bnd_inf
    ub[bnd_key == 'lo'] = bnd_inf
    return bnd_key, lb, ub

def _transform_parameter_with_brcov(u, C, S, F, wb, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd):
    vol_S = np.sqrt(S)
    u_adj = u * (1.0 / vol_S)
    V = np.linalg.cholesky(C).T
    K = V.dot(F) * (1.0 / vol_S)
    #
    eq_matrix_adj = eq_matrix * (1.0 / vol_S)
    eq_bnd_adj = eq_bnd - eq_matrix.dot(wb)
    ineq_matrix_adj = ineq_matrix * (1.0 / vol_S)
    ineq_bnd_adj = ineq_bnd - ineq_matrix.dot(wb)
    #
    lb_adj = (bnd_con[1] - wb) * vol_S
    ub_adj = (bnd_con[2] - wb) * vol_S
    bnd_con = [bnd_con[0], lb_adj, ub_adj]
    return u_adj, K, bnd_con, ineq_matrix_adj, ineq_bnd_adj, eq_matrix_adj, eq_bnd_adj


def _transform_parameter_with_nmcov(cov, wb, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd):
    V = np.linalg.cholesky(cov).T
    #
    eq_bnd_adj = eq_bnd - eq_matrix.dot(wb)
    ineq_bnd_adj = ineq_bnd - ineq_matrix.dot(wb)
    #
    lb_adj = bnd_con[1] - wb
    ub_adj = bnd_con[2] - wb
    bnd_con = [bnd_con[0], lb_adj, ub_adj]
    return V, bnd_con, ineq_bnd_adj, eq_bnd_adj


def _solve_without_turn_with_brcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params):
    lmbd, F, C, S = cov_info
    te_sq = te ** 2
    #
    ineq_matrix, ineq_bnd, eq_matrix, eq_bnd = parse_linear_condition(lin_con)
    u_adj, K, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd = _transform_parameter_with_brcov(
        u, C, S, F, wb, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd)
    #
    L, N = K.shape
    bnd_key, lb, ub = parse_bound_conditions(bnd_con, solver_params.pop('bnd_inf'))
    #
    # Define and solve the CVXPY problem.
    x = cp.Variable(N + L)
    obj = cp.Minimize((lmbd / 2.0) * cp.sum_squares(x[: N]) + (lmbd / 2.0) * cp.sum_squares(x[N:]) -
                      u_adj.T @ x[:N])
    constraints = [
        cp.sum_squares(x[: N]) + cp.sum_squares(x[N:]) <= te_sq,
        x[:N] >= lb,
        x[:N] <= ub,
        K @ x[:N] - x[N:] == 0]
    if ineq_matrix.size != 0:
        constraints.append(ineq_matrix @ x[:N] <= ineq_bnd)
    if eq_matrix.size != 0:
        constraints.append(eq_matrix @ x[:N] == eq_bnd)
    #
    prob = cp.Problem(obj, constraints)
    prob.solve(**solver_params)
    status = prob.status
    is_success = (status == 'optimal' or status == 'optimal_inaccurate')
    w = np.array(x.value[:N]) * (1 / np.sqrt(S)) + wb
    return w, is_success, status


def _solve_with_turn_with_brcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params):
    w0, to, rho = turn_con
    te_sq = te ** 2
    lmbd, F, C, S = cov_info
    #
    ineq_matrix, ineq_bnd, eq_matrix, eq_bnd = parse_linear_condition(lin_con)
    u_adj, K, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd = _transform_parameter_with_brcov(
        u, C, S, F, wb, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd)
    vol_S = np.sqrt(S)
    x0 = (w0 - wb) * vol_S
    L, N = K.shape
    bnd_key, lb, ub = parse_bound_conditions(bnd_con, solver_params.pop('bnd_inf'))
    #
    # Define and solve the CVXPY problem.
    x = cp.Variable(N * 2 + L)
    #
    obj = cp.Minimize((lmbd / 2.0) * cp.sum_squares(x[: N]) + (lmbd / 2.0) * cp.sum_squares(x[2 * N:]) -
                      u_adj.T @ x[:N] + rho * cp.sum(x[N: 2 * N]))
    constraints = [
        cp.sum_squares(x[: N]) + cp.sum_squares(x[2 * N:]) <= te_sq,
        x[:N] >= lb,
        x[:N] <= ub,
        x[:N] - cp.multiply(x[N: 2 * N], vol_S) <= x0,
        -x[:N] - cp.multiply(x[N: 2 * N], vol_S) <= -x0,
        x[N: 2 * N] >= 0,
        cp.sum(x[N: 2 * N]) <= to,
        K @ x[:N] - x[2 * N:] == 0]
    if ineq_matrix.size != 0:
        constraints.append(ineq_matrix @ x[:N] <= ineq_bnd)
    if eq_matrix.size != 0:
        constraints.append(eq_matrix @ x[:N] == eq_bnd)
    #
    prob = cp.Problem(obj, constraints)
    prob.solve(**solver_params)
    status = prob.status
    is_success = (status == 'optimal' or status == 'optimal_inaccurate')
    w = np.array(x.value[:N]) * (1.0 / vol_S) + wb
    return w, is_success, status


def _solve_without_turn_with_nmcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params):
    lmbd, cov = cov_info
    #
    ineq_matrix, ineq_bnd, eq_matrix, eq_bnd = parse_linear_condition(lin_con)
    V, bnd_con, ineq_bnd, eq_bnd = _transform_parameter_with_nmcov(
        cov, wb, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd)
    #
    N = u.shape[0]
    bnd_key, lb, ub = parse_bound_conditions(bnd_con, solver_params.pop('bnd_inf'))
    #
    # Define and solve the CVXPY problem.
    x = cp.Variable(N)
    obj = cp.Minimize((lmbd / 2.0) * cp.sum_squares(V @ x) - u.T @ x)
    constraints = [
        cp.norm2(V @ x[:N]) <= te,
        x >= lb,
        x <= ub
    ]
    if ineq_matrix.size != 0:
        constraints.append(ineq_matrix @ x[:N] <= ineq_bnd)
    if eq_matrix.size != 0:
        constraints.append(eq_matrix @ x[:N] == eq_bnd)
    #
    prob = cp.Problem(obj, constraints)
    prob.solve(**solver_params)
    status = prob.status
    is_success = (status == 'optimal' or status == 'optimal_inaccurate')
    w = np.array(x.value[:N]) + wb
    return w, is_success, status


def _solve_with_turn_with_nmcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params):
    w0, to, rho = turn_con
    lmbd, cov = cov_info
    #
    ineq_matrix, ineq_bnd, eq_matrix, eq_bnd = parse_linear_condition(lin_con)
    V, bnd_con, ineq_bnd, eq_bnd = _transform_parameter_with_nmcov(
        cov, wb, bnd_con, ineq_matrix, ineq_bnd, eq_matrix, eq_bnd)
    x0 = w0 - wb
    N = u.shape[0]
    bnd_key, lb, ub = parse_bound_conditions(bnd_con, solver_params.pop('bnd_inf'))
    #
    # Define and solve the CVXPY problem.
    x = cp.Variable(N * 2)
    #
    obj = cp.Minimize((lmbd / 2.0) * cp.sum_squares(V @ x[:N]) - u.T @ x[:N] + rho * cp.sum(x[N: 2 * N]))
    constraints = [
        cp.norm2(V @ x[:N]) <= te,
        x[:N] >= lb,
        x[:N] <= ub,
        x[:N] - x[N: 2 * N] <= x0,
        -x[:N] - x[N: 2 * N] <= -x0,
        x[N: 2 * N] >= 0.0,
        cp.sum(x[N: 2 * N]) <= to]
    if ineq_matrix.size != 0:
        constraints.append(ineq_matrix @ x[:N] <= ineq_bnd)
    if eq_matrix.size != 0:
        constraints.append(eq_matrix @ x[:N] == eq_bnd)
    #
    prob = cp.Problem(obj, constraints)
    prob.solve(**solver_params)
    status = prob.status
    is_success = (status == 'optimal' or status == 'optimal_inaccurate')
    w = np.array(x.value[:N]) + wb
    return w, is_success, status


WARNING_NOT_PRINTED = True


def solve(u, cov_info, wb, te, lin_con, bnd_con, turn_con, solver_params):
    global WARNING_NOT_PRINTED
    if WARNING_NOT_PRINTED:
        print('  warning::>>cvxpy>>identity cov matrix input may raise error after multiple runs '
              'due to a deep scipy bug.')
        WARNING_NOT_PRINTED = False
    if len(cov_info) == 2:
        if turn_con is None:
            rtn = _solve_without_turn_with_nmcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params)
        else:
            rtn = _solve_with_turn_with_nmcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params)
    elif len(cov_info) == 4:
        if turn_con is None:
            rtn = _solve_without_turn_with_brcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params)
        else:
            rtn = _solve_with_turn_with_brcov(u, cov_info, wb, lin_con, bnd_con, turn_con, te, solver_params)
    else:
        assert False, ' error::>>cvxpy>>socp>>cov_info is unknown.'
    if rtn[2] == 'optimal_inaccurate':
        print('  warning::>>cvxpy>>socp>>the result is optimal but inaccurate, increase max_iters/max_iter may help.')
    return rtn


In [2]:
import cvxpy as cp
import numpy as np
x = cp.Variable(5)
x + np.random.rand(5) , cp.multiply(x , np.random.rand(5))

(Expression(AFFINE, UNKNOWN, (5,)), Expression(AFFINE, UNKNOWN, (5,)))

In [1]:
import numpy as np
import cvxpy as cp

from typing import Literal
from src.factor.optimizer.util import SolverInput , SolveCond , SolveVars
from src.factor.optimizer.util.solver_input import LinearConstraint , BoundConstraint , CovConstraint , TurnConstraint , ShortConstraint
from src.factor.basic.var import SYMBOL_STOCK_LB , SYMBOL_STOCK_UB

input = SolverInput.rand()

DEFAULT_SOLVER_CONFIG : dict[str,dict] = {
    'cvxpy.mosek': {'solver': 'MOSEK'},
    'cvxpy.ecos': {'solver': 'ECOS','max_iters': 200},
    'cvxpy.clarabel': {'solver': 'CLARABEL'},
    'cvxpy.osqp': {'solver': 'OSQP',},
    'cvxpy.scs': {'solver': 'SCS','eps': 1e-6},
    'mosek': {},
    'cvxopt': {'show_progress': False}
}

def try_solve(input : SolverInput , solver : Literal['mosek','ecos','osqp','scs','clarabel'] = 'mosek'):
    solver_params = DEFAULT_SOLVER_CONFIG[f'cvxpy.{solver}']
    # Define and solve the CVXPY problem.
    N = len(input.alpha)

    x = cp.Variable(N)
    objective = -input.alpha.T @ x
    constraints :list = [
        x <= input.bnd_con.ub ,
        x >= input.bnd_con.lb ,
    ]

    if input.cov_con.lmbd or input.cov_con.te:
        if input.cov_con.cov_type == 'model':
            l = cp.Variable(len(input.cov_con.C))
            constraints.append(input.cov_con.F @ (x - input.wb) == l)
            if input.cov_con.lmbd:
                S_sq = np.sqrt(input.cov_con.S)
                objective = objective + \
                    input.cov_con.lmbd / 2.0 * cp.sum_squares(cp.multiply(x - input.wb , S_sq)) + \
                    input.cov_con.lmbd / 2.0 * cp.quad_form(l , input.cov_con.C) 
            if input.cov_con.te:
                constraints.append(cp.sum_squares(cp.multiply(x - input.wb , S_sq)) + 
                               cp.quad_form(l , input.cov_con.C) <= input.cov_con.te ** 2)
        else:
            if input.cov_con.lmbd:
                objective = objective + input.cov_con.lmbd / 2.0 * cp.quad_form(x , input.cov_con.cov)
            if input.cov_con.te:
                constraints.append(cp.quad_form(x , input.cov_con.cov) <= input.cov_con.te ** 2)
    
    if np.any(input.lin_con.type == 'fx'):
        eq_pos = input.lin_con.type == 'fx'
        mat = input.lin_con.A[eq_pos]
        bnd = input.lin_con.lb[eq_pos]
        constraints.append(mat @ x == bnd)

    if np.any(input.lin_con.type != 'fx'):
        up_pos = np.isin(input.lin_con.type,['ra', 'up'])
        lo_pos = np.isin(input.lin_con.type,['ra', 'lo'])
        mat = np.vstack((input.lin_con.A[up_pos], -input.lin_con.A[lo_pos]))
        bnd = np.hstack((input.lin_con.ub[up_pos], input.lin_con.lb[lo_pos]))
        constraints.append(mat @ x <= bnd)

    if input.turn_con.dbl or input.turn_con.rho:
        t = cp.Variable(N)
        constraints.append(t >= 0)
        constraints.append(x - t <= input.w0)
        constraints.append(-x - t <= -input.w0)

        if input.turn_con.dbl:
            constraints.append(cp.sum(t) <= input.turn_con.dbl)
        if input.turn_con.rho:
            objective = objective + input.turn_con.rho * cp.sum(t)  

    if input.short_con.pos or input.short_con.cost:
        s = cp.Variable(N)
        constraints.append(s >= 0)
        constraints.append(x - s >= 0)

        if input.short_con.pos:
            constraints.append(cp.sum(s) <= input.turn_con.dbl)
        if input.short_con.cost:
            objective = objective + input.short_con.cost * cp.sum(s)  

    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve(**solver_params)
    status = prob.status
    is_success = (status == 'optimal' or status == 'optimal_inaccurate')
    w = x.value[:N]
    return w, is_success, status

try_solve(input)

(array([0.37814056, 0.54594357, 0.07591586]), True, 'optimal')

In [5]:
try_solve(input , 'ecos') 

c:\Users\jinmeng\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:354: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add an explicit dependency on ECOS or switch to our new
    default solver, Clarabel, by either not specifying a solver argument
    or specifying ``solver=cp.CLARABEL``.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


(array([0.37814731, 0.54592989, 0.0759228 ]), True, 'optimal')

In [3]:
import numpy as np
import cvxpy as cp
import mosek

from typing import Any , Literal
from src.factor.optimizer.util import SolverInput , SolveCond , SolveVars
from src.factor.basic.var import SYMBOL_INF as INF

solver = 'mosek'
DEFAULT_SOLVER_CONFIG : dict[str,dict] = {
    'cvxpy.mosek': {'solver': 'MOSEK'},
    'cvxpy.ecos': {'solver': 'ECOS','max_iters': 200},
    'cvxpy.clarabel': {'solver': 'CLARABEL'},
    'cvxpy.osqp': {'solver': 'OSQP',},
    'cvxpy.scs': {'solver': 'SCS','eps': 1e-6},
    'mosek': {},
    'cvxopt': {'show_progress': False}
}
solver_params = DEFAULT_SOLVER_CONFIG[f'cvxpy.{solver}']

inp = SolverInput.rand()

class Solver:
    def __init__(self , input : SolverInput , 
                 prob_type : Literal['linprog' , 'quadprog' , 'socp'] = 'socp' ,
                 solver_param : dict = {}):
        self.input = input
        self.prob_type : Literal['linprog' , 'quadprog' , 'socp'] = prob_type
        self.solver_param = solver_param

    def parse_input(self):
        self.alpha    = self.input.alpha
        self.w0       = self.input.w0 if self.input.w0 is not None else np.zeros_like(self.alpha)
        self.wb       = self.input.wb if self.input.wb is not None else np.zeros_like(self.alpha)

        if self.prob_type != 'linprog' and self.input.cov_con and self.input.wb is not None:
            self.cov_con   = self.input.cov_con
        else:
            self.cov_con   = None

        if self.input.turn_con and self.input.w0 is not None:
            self.turn_con  = self.input.turn_con
        else:
            self.turn_con  = None

        if self.input.short_con:
            self.short_con  = self.input.short_con
        else:
            self.short_con  = None

        # variable sequence:
        # num_N , num_T (0 or num_N) , num_S (0 or num_N) , num_L (0 or len(self.F)) , num_Q (0 or 2)
        num_N = len(self.alpha)
        num_T = 0 if not self.conds.turn or not self.turn_con else num_N
        num_S = 0 if not self.conds.short or not self.short_con else num_N
        if (not self.conds.qobj and not self.conds.qcon) or not self.cov_con or self.cov_con.cov_type != 'model':
            num_L = 0
        else: num_L = len(self.cov_con.F)
        if not self.conds.qcon or not self.cov_con or not self.cov_con.te:
            num_Q = 0
        else: num_Q = 2

        self.num_vars = SolveVars(num_N , num_T , num_S , num_L , num_Q)
        return self

    def solve(self , turn = True , qobj = True , qcon = True , short = True):
    # def try_solve(input : SolverInput , solver : Literal['mosek','ecos','osqp','scs','clarabel'] = 'mosek'):
        
        self.conds = SolveCond(turn , qobj , qcon , short)
        self.parse_input()
        # Define and solve the CVXPY problem.
        x = cp.Variable(self.num_vars.N)
        objective = -self.input.alpha.T @ x
        constraints :list = [
            x <= self.input.bnd_con.ub ,
            x >= self.input.bnd_con.lb ,
        ]

        if self.cov_con:
            if self.num_vars.L:
                l = cp.Variable(self.num_vars.L)
                constraints.append(self.cov_con.F @ (x - self.wb) == l)
                if self.cov_con.lmbd:
                    S_sq = np.sqrt(self.cov_con.S)
                    objective = objective + self.cov_con.lmbd / 2.0 * \
                        (cp.sum_squares(cp.multiply(x - self.wb , S_sq)) + cp.quad_form(l , self.cov_con.C) )
                if self.cov_con.te:
                    constraints.append(cp.sum_squares(cp.multiply(x - self.wb , S_sq)) + 
                                cp.quad_form(l , self.cov_con.C) <= self.cov_con.te ** 2)
            else:
                if self.cov_con.lmbd:
                    objective = objective + self.cov_con.lmbd / 2.0 * cp.quad_form(x , self.cov_con.cov)
                if input.cov_con.te:
                    constraints.append(cp.quad_form(x , self.cov_con.cov) <= self.cov_con.te ** 2)
        
        eq_pos = self.input.lin_con.type == 'fx'
        if np.any(eq_pos):
            mat = self.input.lin_con.A[eq_pos]
            bnd = self.input.lin_con.lb[eq_pos]
            constraints.append(mat @ x == bnd)

        up_pos = np.isin(self.input.lin_con.type,['ra', 'up'])
        lo_pos = np.isin(self.input.lin_con.type,['ra', 'lo'])
        if np.any(up_pos) or np.any(lo_pos):
            mat = np.vstack((self.input.lin_con.A[up_pos], -self.input.lin_con.A[lo_pos]))
            bnd = np.hstack((self.input.lin_con.ub[up_pos], self.input.lin_con.lb[lo_pos]))
            constraints.append(mat @ x <= bnd)

        if self.turn_con and self.num_vars.T:
            t = cp.Variable(self.num_vars.T)
            constraints.append(t >= 0)
            constraints.append(x - t <= self.w0)
            constraints.append(-x - t <= -self.w0)

            if self.turn_con.dbl: constraints.append(cp.sum(t) <= self.turn_con.dbl)
            if self.turn_con.rho: objective = objective + self.turn_con.rho * cp.sum(t)  

        if self.short_con and self.num_vars.S:
            s = cp.Variable(self.num_vars.S)
            constraints.append(s >= 0)
            constraints.append(x - s >= 0)

            if self.short_con.pos:  constraints.append(cp.sum(s) <= self.short_con.pos)
            if self.short_con.cost: objective = objective + self.short_con.cost * cp.sum(s)  

        prob = cp.Problem(cp.Minimize(objective), constraints)
        prob.solve(**solver_params)
        status = prob.status
        is_success = (status == 'optimal' or status == 'optimal_inaccurate')
        w = x.value
        return w, is_success, status

a = Solver(inp ,'socp')

In [4]:
a.solve()

(array([2.46788539e-01, 7.53211452e-01, 9.06096106e-09]), True, 'optimal')